<a href="https://colab.research.google.com/github/fickas/salt_marsh/blob/main/salt_marsh_ayushi_Final_Reasearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification for salt marshes for Low,Mid and High Tides for RGB Images



Initial Libraries to Import

Importing Code from Github for ease of usage.

In [ ]:
github_name = 'ayushi0916'
repo_name = 'ImageClassification_UAS'
source_file = 'library.py'
url = f'https://raw.githubusercontent.com/{github_name}/{repo_name}/main/{source_file}'
!rm $source_file
!wget $url
%run -i $source_file

--2023-06-14 14:08:04--  https://raw.githubusercontent.com/ayushi0916/ImageClassification_UAS/main/library.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10800 (11K) [text/plain]
Saving to: ‘library.py’

library.py          100%[===================>]  10.55K  --.-KB/s    in 0s      

2023-06-14 14:08:04 (88.3 MB/s) - ‘library.py’ saved [10800/10800]



Mouting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Need to first set up the data-set to be readable.

*   Here the **folder_path_add** is the link to the image folder in the UAS Data Collection folder of the site that you want to classify the images of.
*   Here the **output_folder_add** is the link to the new folder in the drive created to store the readable images.



In [ ]:
folder_path_add = ""

In [ ]:
output_folder_add = ""

In [ ]:
RGB_Image_To_Readable(folder_path_add,output_folder_add)

## Once the images are converted to readable format the data-sets are created with minimum 20 images each in the folders classified as Regular,Irregular and Other.This step is already done and the folder is shared.

Link to the folder of the data-set with the data from Old Town Hill,Essex Bay and North River

Links:
input_folder_1 = '/content/drive/MyDrive/RGB_15JuneMICA_OTH/Regular_RGB_New'

input_folder_2 = '/content/drive/MyDrive/RGB_15JuneMICA_OTH/Irregular_RGB_New'

input_folder_3 = '/content/drive/MyDrive/RGB_15JuneMICA_OTH/Other_RGB_New'

As 20 images for each classification for training is too less we augment the data by using the below function.

Links of the output folders are as below:

  output_dir_regular = '/content/drive/MyDrive/RGB_15JuneMICA_OTH/Album_RGB_New/Regular'

  output_dir_irregular = '/content/drive/MyDrive/RGB_15JuneMICA_OTH/Album_RGB_New/Irregular'

  output_dir_other = '/content/drive/MyDrive/RGB_15JuneMICA_OTH/Album_RGB_New/Other'

In [ ]:
input_folder_1_link=""
input_folder_2_link=""
input_folder_3_link=""
output_dir1_link=""
output_dir2_link=""
output_dir3_link=""

In [ ]:
Augment_data(input_folder_1_link,input_folder_2_link,input_folder_3_link,output_dir1_link,output_dir2_link,output_dir3_link)

##Cross-verifying the number of images saved in Regular, Irregular and Other

In [ ]:
test_augmentation(output_dir1_link,output_dir2_link,output_dir3_link)

Number of images in augmented folder: 208
Number of images in augmented folder: 208
Number of images in augmented folder: 208


## Defining Inception Size

In [ ]:
inception_size = (299,299)

## Created a extract_features function with input as the directory path to attain a vector with



*   We first define the InceptionV3 model
*   Labels are got from the folder names in the main directory


*   Then we loop through each label which joins the label name with directory path to get label path.
*  Once the label path is got then images in the folder are listed and then loaded


*   Once the image is loaded its resized to be sent to the inceptionV3 model.
*  It is converted to a NumPy array using img_to_array


*  To add dimension to the array we use np.expand_dims
*  To preprocess the input we use preprocess_input


*  Once the prediction on the image is made we flatten out the output using flatten().
*  We then add the extracted features and label to a dict



*   Return Dict


The dirpath below is the path where the augmented data lies.

In [ ]:
dirpath = '/content/drive/MyDrive/RGB_15JuneMICA_OTH/Album_RGB_New'

In [ ]:
feature_dict=extract_features(dirpath)

### XGBoost

*   We first extract the features and the labels got from the inceptionV3 model.
*   Then using the label Encoder encode the labels to numericals.


*   Splitting the features and labels into train and test.
*   Converting the train and test data into the DMatrix format which is optimized for XGBoost training


*   Defining the model parameters
*   Training the model


*   Evaluating and printing the accuracy score of the model.

# Please replace the folder_path variable with the path where you stored the augmented data

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Convert feature dictionary attained from the above function extract_features() to numpy arrays as below
features = np.array([f[0] for f in feature_dict.values()])
labels = np.array([f[1] for f in feature_dict.values()])
print(labels)

# Get the image file names
image_names = list(feature_dict.keys())

# Extract the folder names from the image file names
folder_path = '' # replace with your Google Drive folder path of augmented data

image_folders = [os.path.split(os.path.dirname(os.path.join(root, image_name)))[-1] for image_name in image_names for root, dirs, files in os.walk(folder_path) if image_name in files]

# Create a list of image names with the folder names included
image_names_with_folders = [os.path.join(folder, os.path.basename(image_name)) for folder, image_name in zip(image_folders, image_names)]

# Convert the string labels to numerical labels using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
print(labels)

#0-Other 2- Regular 1-Irregular
print(len(features))
print(len(labels))
print(len(image_names_with_folders))

# Split the data into train and test sets
train_features, test_features, train_labels, test_labels, train_image_names, test_image_names = train_test_split(features, labels, image_names_with_folders, test_size=0.2, random_state=42, stratify=labels)

# Convert the data into DMatrix format, which is optimized for XGBoost training
d_matrix_train = xgb.DMatrix(train_features, label=train_labels)
d_matrix_test = xgb.DMatrix(test_features, label=test_labels)

# Define the XGBoost parameters
params = {
    'n_estimators':30,
    'num_class': 3, # change to the number of classes in your dataset
    'seed': 42,
    'eta': 0.2, # reduce the learning rate to mitigate overfitting
    'max_depth': 4, # limit the maximum tree depth to reduce overfitting
    'min_child_weight': 1, # Controls the minimum number of samples required in a child node to keep splitting it
    'subsample': 0.8, # Controls the fraction of the training instances used for each tree
    'colsample_bytree': 0.8, # Controls the fraction of the features used for each tree
    'reg_alpha': 0, # L1 regularization term on the weights
    'reg_lambda': 1, # L2 regularization term on the weights
    'objective': 'multi:softmax', # set the loss function for multiclass classification
    'eval_metric': 'merror' # set the evaluation metric for multiclass classification
}

# Train the XGBoost model
xgb_clf = xgb.XGBClassifier(**params)
xgb_clf = xgb_clf.fit(train_features, train_labels,
                      early_stopping_rounds=20, # stop training if the validation metric does not improve for 10 rounds
                      eval_set=[(test_features, test_labels)],
                      verbose=False)

# Evaluate the model on the test data
y_pred = xgb_clf.predict(test_features)

y_pred_proba = xgb_clf.predict_proba(test_features)[:,1]

result_df, fancy_df = threshold_results(np.linspace(0,1,16,endpoint=True), test_labels, y_pred_proba,average_value='macro')
result_df


## Accuracy

In [ ]:
accuracy = accuracy_score(test_labels, y_pred)  # Compute accuracy
print("Accuracy:", accuracy)

## Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get the confusion matrix for the test set
cm = confusion_matrix(test_labels, y_pred)

# Define class labels for the plot
class_names = label_encoder.classes_

# Plot the confusion matrix using seaborn
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.show()

## Run classification prediction on entire site.

First we extract the image features using the

In [ ]:
output_path = ""

In [ ]:
features_dict_site = image_feature_site(output_path)

Next we run it on XGBoost and generate a csv file to view it on QGIS

In [ ]:
features = np.array([f[0] for f in features_dict_site.values()])
labels = np.array([f[1] for f in features_dict_site.values()])
print(np.unique(labels))

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(np.array([f[1] for f in feature_dict.values()]))
#['0':'Irregular','2':'Regular','1':'Other']
#[0 1 2]

print(np.unique(labels))
# Evaluate the model on the test data
y_pred = xgb_clf.predict(features)



print(np.unique(y_pred))

# Convert the predicted labels back to their original names
predicted_labels = label_encoder.inverse_transform(y_pred)

# Print the predicted label names
print(np.unique(predicted_labels))

y_pred_proba = xgb_clf.predict_proba(features)[:,1]

import csv
headers = ["Image Number", "Image Address", "Predicted Label"]

# Open the CSV file for writing
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the column headers as the first row in the CSV file
    writer.writerow(headers)
    # Loop through the dictionary and print the image address and predicted label
    for i, (img_address, (features, _)) in enumerate(features_dict_site.items()):
        predicted_label = label_encoder.classes_[y_pred[i]]
        img_filename = img_address.split(".")[0] + ".tif"
        row = [(i),img_filename,predicted_label]
        writer.writerow(row)


Values of how many images got classified under which label

In [ ]:
unique_labels, counts = np.unique(y_pred, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"Label {label}: {count}")